In [1]:
import os
import numpy as np
import pandas as pd

# Get Relevant information from ECLASS file

In [2]:
ECLASS_Complete = pd.read_csv('C:/Users/Cole/Documents/ECLASS_DATA/ECLASS_Merged_Concat.csv')
ECLASS_Complete['Academic Term Sdescr'] = ECLASS_Complete['Year'].astype(str) + ECLASS_Complete['Semester'].apply(lambda x: x[:2].upper())
ECLASS_Complete['Catalog Nbr'] = ECLASS_Complete['Course'].apply(lambda x: x[1:])
ECLASS_Complete['Q3_3_TEXT_x'] = ECLASS_Complete['Q3_3_TEXT_x'].astype(str).str.split('@').str.get(0).str.lower()
ECLASS_Complete['Q3_3_TEXT_y'] = ECLASS_Complete['Q3_3_TEXT_y'].astype(str).str.split('@').str.get(0).str.lower()

df = ECLASS_Complete[['Q3_3_TEXT_x', 'Q3_3_TEXT_y', 'Student_Score_x', 'Student_Score_y', 'Academic Term Sdescr',
                      'Catalog Nbr']].rename(columns = {'Student_Score_x':'PreScores', 'Student_Score_y':'PostScores'})

# Merge with Registrar Data

In [3]:
df_Registrar = pd.read_excel('Registrar_Data.xls').drop(columns = ['ACT | Combined English/Writing', 'ACT | English', 
                                                                   'ACT | Reading', 'ACT | Writing',
                                                                   'ACT | Writing Subject Score 9/15', 'SAT I | Critical Reading', 
                                                                   'SAT I | Read/Writing Sect Score', 'SAT I | Writing Score'])
df_Registrar[['Catalog Nbr', 'Employee Id']] = df_Registrar[['Catalog Nbr', 'Employee Id']].astype(str)
df_Registrar['Academic_Term_Number'] = df_Registrar['Academic Term Sdescr'].map({'2017FA':1, '2018SP':2, '2018FA':3, '2019SP':4})
df_Registrar = df_Registrar.sort_values(by = 'Academic_Term_Number').drop_duplicates(subset = ['Employee Id', 
                                                                                               'Catalog Nbr'], keep = 'last')

df1 = pd.merge(df, df_Registrar, left_on = ['Q3_3_TEXT_x', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Netid', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df2 = pd.merge(df, df_Registrar, left_on = ['Q3_3_TEXT_x', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Employee Id', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df3 = df.merge(df_Registrar, left_on = ['Q3_3_TEXT_y', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Netid', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df4 = df.merge(df_Registrar, left_on = ['Q3_3_TEXT_y', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Employee Id', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')

df_merged = pd.concat([df1, df2, df3, df4]).drop_duplicates(subset = ['Employee Id', 'Academic Term Sdescr'],
                                                            keep = 'last').reset_index(drop = True)

index_vals = [tuple(v) for v in df_merged[['Employee Id', 'Academic Term Sdescr']].values]
filtered = df_Registrar.loc[(df_Registrar['Academic Term Sdescr'] + 
                             df_Registrar['Catalog Nbr']).isin(df_merged['Academic Term Sdescr'] + 
                                                               df_merged['Catalog Nbr'])].set_index(['Employee Id',
                                                                                                     'Academic Term Sdescr'])
filtered_out = filtered.loc[~filtered.index.isin(index_vals)].reset_index()

df_out = pd.concat([df_merged, filtered_out]).drop(columns = ['Q3_3_TEXT_x', 'Q3_3_TEXT_y']).reset_index(drop = True)
df_out = df_out.loc[:, list(df_Registrar.columns) + ['PreScores', 'PostScores']]
df_out['Assessment'] = 'ECLASS'
df_out.to_csv('ECLASSpRegistrar.csv', index = False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

